In [ ]:
# Connect to the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install the necessary packages
!pip install segmentation_models
!pip install keras_applications==1.0.7
!pip install efficientnet==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: keras_applications
    Found existing installation: Keras-Applications 1.0.8
    Uninstalling Keras-Applications-1.0.8:
      Successfully uninstalled Keras-Applications-1.0.8


In [ ]:
# change SM framework to tensorflow keras
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

# Set channels first, since the channels are the first dimension ( (2, 416, 704) for input, and (3, 416, 704) for output)
keras.backend.set_image_data_format('channels_first')

Segmentation Models: using `tf.keras` framework.


In [ ]:
# Import libraries
from segmentation_models import Linknet, Unet, FPN
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
import keras
import os
import pandas as pd
import numpy as np

In [ ]:
img_dir = '' # Give location of all the input data
files = os.listdir(img_dir)

data = pd.read_csv(img_dir + "/" + files[0])
input_data = data.Y

mean_y = np.mean(input_data)
std_y  = np.std(input_data)

In [ ]:
# define custom dataset class
# For data loading during model train and testing
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

import numpy as np
import os
import pandas as pd
from torchvision.io import read_image

from sklearn.preprocessing import MinMaxScaler


class CustomImageDataset(Dataset):
    def __init__(self, label_dir, img_dir, annotation_dir, transform=None, target_transform=None):
        self.label_dir = label_dir
        self.annotations = pd.read_csv(annotation_dir, delimiter = ';') # to dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.target_shape = (416, 704)
    def __len__(self):
        return len(os.listdir(self.label_dir))-2
        #return len(self.img_labels)

    def __getitem__(self, idx):
        # input data
        input_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        input = pd.read_csv(input_path)
        image = (input.Y-mean_y)/std_y
        image = pd.DataFrame(image.values.reshape(401, 701))
        image = np.pad(image, ((0, 15), (0, 3)), mode='constant')
        wind_inlet = input.inlet_wind_normal_surf_diff
        wind_inlet = pd.DataFrame(wind_inlet.values.reshape(401, 701))
        wind_inlet = np.pad(wind_inlet, ((0, 15), (0, 3)), mode='constant')
        input_data = np.stack((image, wind_inlet))

        # output data
        label_path = os.path.join(self.label_dir, self.annotations.iloc[idx, 1])
        label = pd.read_csv(label_path)

        U_0 = label.loc[:, "U_0"]
        U_0 = (U_0-np.mean(U_0))/np.std(U_0)

        U_1 = label.loc[:, "U_1"]
        U_1 = (U_1-np.mean(U_1))/np.std(U_1)

        U_2 = label.loc[:, "U_2"]
        U_2 = (U_2-np.mean(U_2))/np.std(U_2)

        U_0 = pd.DataFrame(U_0.values.reshape(401, 701))
        U_1 = pd.DataFrame(U_1.values.reshape(401, 701))
        U_2 = pd.DataFrame(U_2.values.reshape(401, 701))

        U_0 = np.pad(U_0, ((0, 15), (0, 3)), mode='constant')
        U_1 = np.pad(U_1, ((0, 15), (0, 3)), mode='constant')
        U_2 = np.pad(U_2, ((0, 15), (0, 3)), mode='constant')

        output_data = np.stack((U_0, U_1, U_2))

        return input_data, output_data

In [ ]:
# Define custom dataloader class
# for our specific data
from torch.utils.data import DataLoader
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches

    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """

    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):

        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        return batch

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [ ]:
# define cumtom dataset for the test set
class CustomImageDataset_test(Dataset):
    def __init__(self, label_dir, img_dir, transform=None, target_transform=None):
        self.label_dir = label_dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.target_shape = (416, 704)
    def __len__(self):
        return 1
        #return len(self.img_labels)

    def __getitem__(self, idx):
        # input data
        input = pd.read_csv(self.img_dir)
        image = (input.Y-mean_y)/std_y
        image = pd.DataFrame(image.values.reshape(401, 701))
        image = np.pad(image, ((0, 15), (0, 3)), mode='constant')
        wind_inlet = input.inlet_wind_normal_surf_diff
        wind_inlet = pd.DataFrame(wind_inlet.values.reshape(401, 701))
        wind_inlet = np.pad(wind_inlet, ((0, 15), (0, 3)), mode='constant')
        input_data = np.stack((image, wind_inlet))

        label = pd.read_csv(self.label_dir)

        # output data
        U_0 = label.loc[:, "U_0"]
        U_0 = (U_0-np.mean(U_0))/np.std(U_0)

        U_1 = label.loc[:, "U_1"]
        U_1 = (U_1-np.mean(U_1))/np.std(U_1)

        U_2 = label.loc[:, "U_2"]
        U_2 = (U_2-np.mean(U_2))/np.std(U_2)

        U_0 = pd.DataFrame(U_0.values.reshape(401, 701))
        U_1 = pd.DataFrame(U_1.values.reshape(401, 701))
        U_2 = pd.DataFrame(U_2.values.reshape(401, 701))

        U_0 = np.pad(U_0, ((0, 15), (0, 3)), mode='constant')
        U_1 = np.pad(U_1, ((0, 15), (0, 3)), mode='constant')
        U_2 = np.pad(U_2, ((0, 15), (0, 3)), mode='constant')

        output_data = np.stack((U_0, U_1, U_2))

        return input_data, output_data

In [ ]:
# make the custom dataset
label_dir = '' # Give location of the output (target) data
img_dir = '' # Give location of the input data

annotation_dir = '' # Give the location of the annotation file

# define the dataset
dataset = CustomImageDataset(label_dir, img_dir, annotation_dir)

In [ ]:
annotations = pd.read_csv(annotation_dir, delimiter = ';')
annotations.head()

,input_file,output_file
0,Copy of angle_-10_inlet_wind_normal_surf_diff.csv,Copy of angle_-10_export_postprocessing_U_DEM_...
1,Copy of angle_-40_inlet_wind_normal_surf_diff.csv,Copy of angle_-40_export_postprocessing_U_DEM_...
2,Copy of angle_-60_inlet_wind_normal_surf_diff.csv,Copy of angle_-60_export_postprocessing_U_DEM_...
3,Copy of angle_-70_inlet_wind_normal_surf_diff.csv,Copy of angle_-70_export_postprocessing_U_DEM_...
4,Copy of angle_0_inlet_wind_normal_surf_diff.csv,Copy of angle_0_export_postprocessing_U_DEM_be...


In [ ]:
# Extract the angle of the test fold
# For generic running of the code
import re
def extract_angle(s):
    # Define the regular expression pattern to match the angle degrees
    pattern = r'angle_(-?\d+)_'

    # Search for the pattern in the string
    match = re.search(pattern, s)

    if match:
        # Extract the angle value and convert it to an integer
        angle = int(match.group(1))
        return angle
    else:
        raise ValueError("No valid angle found in the string")

In [ ]:
# define test and validation inlet angles (indexes of the annotation file)
test_indices = [[0, 3], [2, 5], [9, 11], [1, 7], [8, 10]]
val_indices = [4, 6, 12]

In [ ]:
# Create new directory to store all the results and the best model
directory = ''

# Define the names of the folders for each fold that will be created
angle_dirs = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']

In [ ]:
# Train and test the CNN model

from torch.utils.data import Subset
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import time

# Define metric and loss function
metric = keras.metrics.RootMeanSquaredError()
loss = keras.losses.MeanSquaredError()

# Define backbone
backbone = 'efficientnetb7' #'resnet152' #'densenet121'

for count, test_pair in enumerate(test_indices[3:]):

  model_dir = directory + angle_dirs[count+3]

  callbacks = [
    keras.callbacks.ModelCheckpoint(model_dir + '/best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
  ]

  # Get the 2 test angles
  output_dir = annotations.output_file[test_pair[0]]
  input_dir = annotations.input_file[test_pair[0]]
  test_label = label_dir + "/" + output_dir
  angle1 = extract_angle(test_label)

  output_dir = annotations.output_file[test_pair[1]]
  input_dir = annotations.input_file[test_pair[1]]
  test_label = label_dir + "/" + output_dir
  angle2 = extract_angle(test_label)
  angles = [angle1, angle2]



  # Define test set
  test_set = Subset(dataset, test_pair)

  # Define validation set
  validation_set = Subset(dataset, val_indices)

  # Define train set
  exclude = test_pair + val_indices
  train_indices = np.arange(0, 13)            # Total of 13 datapoints
  train_indices = np.delete(train_indices, exclude) # Exclude the test and validation datapoints from training
  train_set = Subset(dataset, train_indices)

  # Define loaders
  train_loader = Dataloder(dataset = train_set, batch_size = 2, shuffle=True)
  validation_loader = Dataloder(dataset = validation_set)
  test_loader = Dataloder(dataset = test_set)

  # Define & compile the model
  model = Linknet(backbone_name=backbone, encoder_weights=None, input_shape=(2, None, None), classes = 3, activation = 'linear') # Here you can adjust the type of CNN architecture
  model.compile('Adam', loss=loss, metrics=[metric])

  start_time = time.time()
  # Train the model with datapoint i left out
  history = model.fit_generator(
    train_loader,
    steps_per_epoch=len(train_loader),
    epochs=75,
    callbacks=callbacks,
    validation_data=validation_loader,
    validation_steps=len(validation_loader)
  )
  end_time = time.time()
  train_time = end_time - start_time

  # save train time
  with open(model_dir + f'/train_time_test_angles_{angle1}_{angle2}.txt', 'w') as f:
      f.write(str(train_time))

  ## plot training graph
  import matplotlib.pyplot as plt

  # Plot both root_mean_squared_error and loss in the same figure
  plt.figure(figsize=(16, 8))

  # Plot root_mean_squared_error
  plt.subplot(121)
  plt.plot(history.history['root_mean_squared_error'])
  plt.plot(history.history['val_root_mean_squared_error'])  # Add validation RMSE
  plt.title('Model RMSE score')
  plt.ylabel('RMSE')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  #plt.ylim(0, 10)  # Set y-axis limit to 70

  # Plot loss (MSE)
  plt.subplot(122)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])  # Add validation loss
  plt.title('Model loss')
  plt.ylabel('Loss (MSE)')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper right')
  #plt.ylim(0, 90)  # Set y-axis limit to 70
  plt.savefig(model_dir + f'/Training_graph_test_angles_{angle1}_{angle2}.png') #{angle1} & {angle2}.png')
  plt.show()

  # save raw data of the training graph
  training_data = {
      'rmse': history.history['root_mean_squared_error'],
      'val_rmse': history.history['val_root_mean_squared_error'],
      'loss': history.history['loss'],
      'val_loss': history.history['val_loss']
  }

  training_graph_data = pd.DataFrame(training_data)
  training_graph_data.to_csv(model_dir + f'/Training_graph_data_test_angles_{angle1}_{angle2}.csv')

  # Load the best model
  model.load_weights(model_dir + '/best_model.h5')

  # predict, evaluate and plot the testset
  for i in range(len(test_pair)):
    # Extract the angle of the fold
    angle = angles[i]

    # Compute the std and mean of the U_i to get the orgininal values
    label_path = os.path.join(label_dir, annotations.iloc[test_pair[i], 1])
    label = pd.read_csv(label_path)
    U_0 = label.loc[:, "U_0"]
    mean_0 = np.mean(U_0)
    std_0  = np.std(U_0)

    U_1 = label.loc[:, "U_1"]
    mean_1 = np.mean(U_1)
    std_1  = np.std(U_1)

    U_2 = label.loc[:, "U_2"]
    mean_2 = np.mean(U_2)
    std_2  = np.std(U_2)

    # Predict on the correct testset
    prediction = model.predict(test_loader[i][0])

    # Rescale the predictions and ground truth data
    ground_truth_0 = test_loader[i][1][0][0][:-15, :-3]*std_0 + mean_0
    ground_truth_1 = test_loader[i][1][0][1][:-15, :-3]*std_1 + mean_1
    ground_truth_2 = test_loader[i][1][0][2][:-15, :-3]*std_2 + mean_2 # probably needs and extra [0] between [1] and [2]
    ground_truths = [ground_truth_0, ground_truth_1, ground_truth_2]

    prediction_0 = prediction[0][0][:-15, :-3]*std_0 + mean_0
    prediction_1 = prediction[0][1][:-15, :-3]*std_1 + mean_1
    prediction_2 = prediction[0][2][:-15, :-3]*std_2 + mean_2
    predictions = [prediction_0, prediction_1, prediction_2]

    # Compute model performance with fold i
    actual_rmse_value_0 = np.sqrt(mean_squared_error(ground_truth_0, prediction_0))
    actual_rmse_value_1 = np.sqrt(mean_squared_error(ground_truth_1, prediction_1))
    actual_rmse_value_2 = np.sqrt(mean_squared_error(ground_truth_2, prediction_2))
    actual_rmse = [actual_rmse_value_0, actual_rmse_value_1, actual_rmse_value_2]

    # Compute model performance with fold i
    standardized_rmse_value_0 = np.sqrt(mean_squared_error(test_loader[i][1][0][0][:-15, :-3], prediction[0][0][:-15, :-3]))
    standardized_rmse_value_1 = np.sqrt(mean_squared_error(test_loader[i][1][0][1][:-15, :-3], prediction[0][1][:-15, :-3]))
    standardized_rmse_value_2 = np.sqrt(mean_squared_error(test_loader[i][1][0][2][:-15, :-3], prediction[0][2][:-15, :-3]))
    standardized_rmse = [standardized_rmse_value_0, standardized_rmse_value_1, standardized_rmse_value_2]

    ## Save the different RMSE values in a txt file
    with open(model_dir + f'/actual_rmse_test_angle_{angle}.txt', 'w') as f:
        f.write(f"RMSE_0: {actual_rmse_value_0:.4f}\n")
        f.write(f"RMSE_1: {actual_rmse_value_1:.4f}\n")
        f.write(f"RMSE_2: {actual_rmse_value_2:.4f}\n")

    ## Save the different RMSE values in a txt file
    with open(model_dir + f'/standardized_rmse_test_angle_{angle}.txt', 'w') as f:
        f.write(f"RMSE_0: {standardized_rmse_value_0:.4f}\n")
        f.write(f"RMSE_1: {standardized_rmse_value_1:.4f}\n")
        f.write(f"RMSE_2: {standardized_rmse_value_2:.4f}\n")

    # Used to keep prediction and True velocity values on the same axis
    levels_U_0 = np.linspace(min(ground_truth_0.min(), prediction_0.min()), max(ground_truth_0.max(), prediction_0.max()), 100)
    levels_U_1 = np.linspace(min(ground_truth_1.min(), prediction_1.min()), max(ground_truth_1.max(), prediction_1.max()), 100)
    levels_U_2 = np.linspace(min(ground_truth_2.min(), prediction_2.min()), max(ground_truth_2.max(), prediction_2.max()), 100)
    levels = [levels_U_0, levels_U_1, levels_U_2]

    ## Plot and save the different graphs
    fig, axes = plt.subplots(3, 3, figsize=(18, 8), sharex=True, sharey=True)  # Create a figure with 3 subplots
    for j in range(3):
        # plot ground truths
        plot_values = ground_truths[j]
        ax = axes[0, j]
        p1 = ax.contourf(plot_values, levels=levels[j], cmap='jet')
        fig.colorbar(p1, ax=ax)
        ax.title.set_text(f'Ground truth U_{j} of angle {angle}')
        ax.set_xlabel('X-axis')
        ax.set_ylabel('Z-axis')

        # plot predictions
        plot_values = predictions[j]
        ax = axes[1, j]
        p2 = ax.contourf(plot_values, levels=levels[j], cmap='jet')
        fig.colorbar(p2, ax=ax)
        ax.set_title(f'Prediction U_{j-3} of angle {angle}')
        ax.set_xlabel('X-axis')
        ax.set_ylabel('Z-axis')

        # plot errors
        plot_values = predictions[j] - ground_truths[j]
        ax = axes[2, j]
        p3 = ax.contourf(plot_values, levels=100, cmap='jet')
        fig.colorbar(p3, ax=ax)
        ax.set_title(f'Error U_{j} of angle {angle}, RMSE: {actual_rmse[j]:.2f} m/s')
        ax.set_xlabel('X-axis')
        ax.set_ylabel('Z-axis')

    plt.tight_layout()  # Adjust layout to prevent overlap

    # Save the graph
    plt.savefig(model_dir + f'/Graph_test_angle_{angle}.png')

Output hidden; open in https://colab.research.google.com to view.